In [14]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from matplotlib import mlab
from scipy import signal, stats
from sklearn import preprocessing
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from datetime import datetime
from datetime import timedelta
import jpholiday
from plotly.subplots import make_subplots

Tokyo_station_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Tokyostation.npy"
Tokyo_station_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Tokyostaion.npy"

Kyoto_station_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Kyotostation.npy"
Kyoto_station_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Kyotostation.npy"

Arashi_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Arashiyama.npy"
Arashi_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Arashiyama.npy"

Karasuma_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Karasuma.npy"
Karasuma_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Karasuma.npy"

Kinkaku_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Kinkaku.npy"
Kinkaku_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Kinkaku.npy"

Kiyomizu_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Kiyomizu.npy"
Kiyomizu_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Kiyomizu.npy"

University_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/University.npy"
University_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/University.npy"

Nizyou_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Nizyou.npy"
Nizyou_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Nizyou.npy"

Touzi_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/numpy_array/Touzi.npy"
Touzi_tweets_PATH = "//home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/numpy_array/Touzi.npy"

list_mobile = [
    Tokyo_station_mobile_PATH,
    Arashi_mobile_PATH,
    Karasuma_mobile_PATH,
    Kinkaku_mobile_PATH,
    Kiyomizu_mobile_PATH,
    University_mobile_PATH,
    Nizyou_mobile_PATH,
    Touzi_mobile_PATH,
    Kyoto_station_mobile_PATH,
]
list_twitter = [
    Tokyo_station_tweets_PATH,
    Arashi_tweets_PATH,
    Karasuma_tweets_PATH,
    Kinkaku_tweets_PATH,
    Kiyomizu_tweets_PATH,
    University_tweets_PATH,
    Nizyou_tweets_PATH,
    Touzi_tweets_PATH,
    Kyoto_station_tweets_PATH,
]
list_key = [
    "Tokyo Station",
    "Arashiyama",
    "Karasuma",
    "Kinkaku Temple",
    "Kiyomizu Temple",
    "Kyoto University",
    "Nizyou",
    "Touzi",
    "Kyoto Station",
]

In [15]:
def isBizDay(DATE):
    # DATE = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    if DATE.weekday() >= 5 or jpholiday.is_holiday(DATE):
        return "Holiday"
    else:
        return "Workday"

In [16]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        data.append([str(i) + '時', correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
fig.update_layout(title='1時間ごとのソーシャルセンサ性能 (平日)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='時間'),
                 yaxis=dict(title='ソーシャルセンサ性能'))
fig.show()

/home/is/shuntaro-o/dev/compare_population_and_tweet_number/.venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



In [17]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
#fig.update_layout(title='Hourly social sensor performance (workday)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='hour'),
                 yaxis=dict(title='social sensor performance'))
fig.update_layout(width=700,height=500)
fig.show()

In [18]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        data.append([str(i) + '時', correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
fig.update_layout(title='1時間ごとのソーシャルセンサ性能 (休日)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='時間'),
                 yaxis=dict(title='ソーシャルセンサ性能'))
fig.show()

In [19]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
#fig.update_layout(title='Hourly social sensor performance (holiday)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='hour'),
                 yaxis=dict(title='social sensor performance'))
fig.update_layout(width=700,height=500)
fig.show()

In [20]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df_work= pd.DataFrame(data,columns=columns)
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']
color=['red','orange','green','aquamarine','blue','aqua','purple','pink','lime']

df_holi= pd.DataFrame(data,columns=columns)

fig = make_subplots(rows=1, cols=2,subplot_titles=['Workday','Holiday'], shared_yaxes='all', shared_xaxes='all')

list_place = df_work['place'].unique()
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[0]]['time'], y = df_work[df_work['place']==list_place[0]]['r'], name = 'Tokyo Station', line=dict(color=color[0])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[1]]['time'], y = df_work[df_work['place']==list_place[1]]['r'], name = list_place[1],line=dict(color=color[1])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[2]]['time'], y = df_work[df_work['place']==list_place[2]]['r'], name = 'Nakagyou',line=dict(color=color[2])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[3]]['time'], y = df_work[df_work['place']==list_place[3]]['r'], name = 'Kinkaku Temple',line=dict(color=color[3])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[4]]['time'], y = df_work[df_work['place']==list_place[4]]['r'], name = 'Kiyomizu Temple',line=dict(color=color[4])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[5]]['time'], y = df_work[df_work['place']==list_place[5]]['r'], name = 'Kyoto University',line=dict(color=color[5])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[6]]['time'], y = df_work[df_work['place']==list_place[6]]['r'], name = 'Nizyou Castle ',line=dict(color=color[6])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[7]]['time'], y = df_work[df_work['place']==list_place[7]]['r'], name = 'Touzi Temple',line=dict(color=color[7])),row=1,col=1)
fig.add_trace(go.Scatter(x = df_work[df_work['place']==list_place[8]]['time'], y = df_work[df_work['place']==list_place[8]]['r'], name = 'Kyoto Station',line=dict(color=color[8])),row=1,col=1)

fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[0]]['time'], y = df_holi[df_holi['place']==list_place[0]]['r'], name = list_place[0],showlegend = False,line=dict(color=color[0])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[1]]['time'], y = df_holi[df_holi['place']==list_place[1]]['r'], name = list_place[1],showlegend = False,line=dict(color=color[1])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[2]]['time'], y = df_holi[df_holi['place']==list_place[2]]['r'], name = list_place[2],showlegend = False,line=dict(color=color[2])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[3]]['time'], y = df_holi[df_holi['place']==list_place[3]]['r'], name = list_place[3],showlegend = False,line=dict(color=color[3])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[4]]['time'], y = df_holi[df_holi['place']==list_place[4]]['r'], name = list_place[4],showlegend = False,line=dict(color=color[4])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[5]]['time'], y = df_holi[df_holi['place']==list_place[5]]['r'], name = list_place[5],showlegend = False,line=dict(color=color[5])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[6]]['time'], y = df_holi[df_holi['place']==list_place[6]]['r'], name = list_place[6],showlegend = False,line=dict(color=color[6])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[7]]['time'], y = df_holi[df_holi['place']==list_place[7]]['r'], name = list_place[7],showlegend = False,line=dict(color=color[7])),row=1,col=2)
fig.add_trace(go.Scatter(x = df_holi[df_holi['place']==list_place[8]]['time'], y = df_holi[df_holi['place']==list_place[8]]['r'], name = list_place[8],showlegend = False,line=dict(color=color[8])),row=1,col=2)
#fig.update_layout(title='Hourly social sensor performance (workday)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_xaxes(title='hour', row=1, col=1)
fig.update_xaxes(title='hour', row=1, col=2)
fig.update_yaxes(title='social sensor performance', row=1, col=1)


fig.update_layout(width=1400,height=500)
fig.show()

In [82]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)
    tmp= [] 
    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        tmp.append('{:.2f} ({:.2f})'.format(correlation,p_value))
    data.append(tmp)



columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

data = np.array(data)
df_work= pd.DataFrame(data,index=df['place'].unique())

data = []
tmp = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    tmp = []
    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        tmp.append('{:.2f} ({:.2f})'.format(correlation,p_value))
    data.append(tmp)




columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

data = np.array(data)
df_holi= pd.DataFrame(data, index=df['place'].unique())

/home/is/shuntaro-o/dev/compare_population_and_tweet_number/.venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/home/is/shuntaro-o/dev/compare_population_and_tweet_number/.venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



In [83]:
df_work

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Tokyo Station,0.03 (0.59),0.07 (0.29),-0.07 (0.30),-0.01 (0.85),-0.05 (0.44),0.22 (0.00),0.46 (0.00),0.29 (0.00),0.29 (0.00),0.16 (0.01),...,0.28 (0.00),0.32 (0.00),0.31 (0.00),0.39 (0.00),0.52 (0.00),0.50 (0.00),0.53 (0.00),0.59 (0.00),0.53 (0.00),0.47 (0.00)
Arashiyama,0.10 (0.11),0.04 (0.58),-0.05 (0.43),0.11 (0.10),0.00 (0.94),0.08 (0.23),-0.02 (0.77),0.16 (0.01),0.10 (0.12),0.27 (0.00),...,0.37 (0.00),0.30 (0.00),0.11 (0.08),0.07 (0.29),0.16 (0.01),0.15 (0.02),0.04 (0.54),0.06 (0.34),0.06 (0.32),0.14 (0.03)
Karasuma,-0.05 (0.46),-0.02 (0.70),-0.00 (0.98),-0.05 (0.43),-0.04 (0.58),0.02 (0.72),-0.04 (0.49),0.03 (0.64),0.09 (0.17),0.03 (0.63),...,0.00 (0.98),0.03 (0.68),-0.04 (0.57),0.05 (0.41),0.02 (0.72),0.04 (0.53),0.07 (0.28),0.13 (0.04),0.07 (0.27),0.06 (0.38)
Kinkaku Temple,0.01 (0.90),-0.00 (0.98),-0.05 (0.47),0.09 (0.14),0.04 (0.56),-0.01 (0.92),0.08 (0.22),0.09 (0.18),0.00 (0.99),0.09 (0.18),...,0.02 (0.78),0.11 (0.09),0.11 (0.09),0.05 (0.47),0.06 (0.36),-0.12 (0.06),0.10 (0.11),-0.14 (0.03),-0.09 (0.17),-0.14 (0.02)
Kiyomizu Temple,0.08 (0.19),0.07 (0.31),0.02 (0.77),0.02 (0.81),-0.13 (0.04),0.02 (0.78),0.09 (0.16),0.18 (0.00),0.20 (0.00),-0.06 (0.35),...,0.25 (0.00),0.27 (0.00),0.27 (0.00),0.32 (0.00),0.25 (0.00),0.45 (0.00),0.35 (0.00),0.33 (0.00),0.22 (0.00),0.17 (0.01)
Kyoto University,-0.01 (0.93),nan (nan),-0.05 (0.40),0.03 (0.65),-0.11 (0.09),0.15 (0.02),-0.04 (0.49),0.04 (0.57),0.06 (0.37),-0.00 (0.96),...,0.12 (0.06),-0.19 (0.00),0.03 (0.67),0.01 (0.91),0.06 (0.34),-0.02 (0.70),0.04 (0.58),-0.02 (0.77),0.00 (0.97),-0.00 (0.96)
Nizyou,0.17 (0.01),-0.05 (0.46),-0.02 (0.71),0.04 (0.56),0.05 (0.48),-0.03 (0.68),0.04 (0.56),0.05 (0.44),0.04 (0.49),0.14 (0.03),...,0.09 (0.15),0.04 (0.52),0.09 (0.14),0.11 (0.08),0.18 (0.01),0.08 (0.23),0.18 (0.00),0.02 (0.72),0.18 (0.01),0.20 (0.00)
Touzi,0.09 (0.17),0.05 (0.41),0.04 (0.57),0.04 (0.50),-0.07 (0.26),-0.07 (0.31),0.08 (0.19),0.10 (0.14),0.15 (0.02),0.06 (0.34),...,-0.04 (0.56),0.04 (0.51),0.07 (0.31),0.08 (0.21),0.12 (0.06),0.21 (0.00),0.10 (0.11),0.05 (0.44),-0.00 (0.97),0.10 (0.11)
Kyoto Station,0.39 (0.00),0.08 (0.20),0.17 (0.01),0.18 (0.01),0.13 (0.04),0.08 (0.19),0.30 (0.00),0.40 (0.00),0.44 (0.00),0.40 (0.00),...,0.43 (0.00),0.50 (0.00),0.53 (0.00),0.55 (0.00),0.61 (0.00),0.55 (0.00),0.66 (0.00),0.64 (0.00),0.59 (0.00),0.54 (0.00)


In [84]:
df_holi

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Tokyo Station,0.21 (0.02),-0.03 (0.72),-0.11 (0.21),-0.19 (0.04),-0.06 (0.52),0.40 (0.00),0.82 (0.00),0.79 (0.00),0.66 (0.00),0.67 (0.00),...,0.57 (0.00),0.57 (0.00),0.56 (0.00),0.53 (0.00),0.53 (0.00),0.60 (0.00),0.52 (0.00),0.52 (0.00),0.40 (0.00),0.50 (0.00)
Arashiyama,-0.05 (0.59),0.06 (0.48),0.17 (0.07),nan (nan),-0.00 (0.96),0.09 (0.34),0.03 (0.76),0.01 (0.89),0.32 (0.00),0.20 (0.03),...,0.58 (0.00),0.34 (0.00),0.22 (0.02),0.01 (0.87),0.32 (0.00),0.10 (0.28),0.09 (0.30),0.13 (0.17),-0.11 (0.23),0.13 (0.16)
Karasuma,0.10 (0.27),0.01 (0.91),-0.01 (0.95),0.21 (0.02),0.01 (0.95),0.03 (0.72),-0.04 (0.64),0.12 (0.18),0.11 (0.21),0.04 (0.64),...,0.17 (0.05),0.08 (0.38),-0.02 (0.85),0.02 (0.79),0.15 (0.10),0.13 (0.14),0.02 (0.82),0.28 (0.00),0.05 (0.58),0.17 (0.06)
Kinkaku Temple,-0.19 (0.03),-0.18 (0.04),0.02 (0.80),-0.05 (0.59),-0.01 (0.92),nan (nan),-0.02 (0.81),-0.11 (0.21),-0.16 (0.09),-0.02 (0.85),...,0.21 (0.02),0.25 (0.01),0.12 (0.17),-0.12 (0.20),-0.17 (0.06),-0.01 (0.90),-0.09 (0.34),-0.05 (0.57),-0.05 (0.59),-0.07 (0.46)
Kiyomizu Temple,-0.04 (0.64),0.03 (0.77),0.13 (0.15),0.01 (0.93),0.04 (0.67),-0.03 (0.78),0.11 (0.22),0.28 (0.00),0.05 (0.56),0.18 (0.04),...,0.31 (0.00),0.32 (0.00),0.49 (0.00),0.48 (0.00),0.68 (0.00),0.47 (0.00),0.53 (0.00),0.29 (0.00),0.30 (0.00),0.01 (0.94)
Kyoto University,0.03 (0.71),-0.04 (0.64),-0.20 (0.03),-0.04 (0.67),nan (nan),-0.08 (0.37),0.01 (0.88),-0.06 (0.50),0.00 (1.00),0.09 (0.30),...,0.04 (0.65),0.03 (0.78),0.03 (0.71),-0.04 (0.65),-0.10 (0.27),0.01 (0.87),-0.03 (0.72),0.08 (0.38),-0.14 (0.12),0.01 (0.92)
Nizyou,-0.04 (0.65),0.10 (0.29),0.05 (0.55),0.01 (0.90),-0.00 (0.96),-0.00 (1.00),-0.07 (0.45),-0.08 (0.40),0.04 (0.70),0.19 (0.03),...,0.09 (0.31),0.10 (0.27),0.23 (0.01),-0.03 (0.71),0.17 (0.07),0.21 (0.02),0.17 (0.06),0.08 (0.39),0.14 (0.11),0.07 (0.42)
Touzi,-0.06 (0.54),0.08 (0.40),-0.03 (0.75),0.04 (0.66),0.08 (0.41),0.11 (0.21),0.18 (0.05),0.08 (0.38),0.08 (0.38),0.23 (0.01),...,0.01 (0.91),-0.09 (0.33),0.07 (0.47),0.13 (0.16),0.20 (0.03),0.36 (0.00),0.28 (0.00),0.04 (0.65),-0.09 (0.33),0.26 (0.00)
Kyoto Station,0.39 (0.00),0.22 (0.02),0.10 (0.25),0.26 (0.00),0.13 (0.17),0.29 (0.00),0.35 (0.00),0.43 (0.00),0.50 (0.00),0.40 (0.00),...,0.44 (0.00),0.38 (0.00),0.47 (0.00),0.42 (0.00),0.45 (0.00),0.47 (0.00),0.46 (0.00),0.40 (0.00),0.51 (0.00),0.44 (0.00)


In [85]:
#df_work.to_csv('/home/is/shuntaro-o/dev/compare_population_and_tweet_number/outputs/perhour/table_workday.csv')

In [86]:
#df_holi.to_csv('/home/is/shuntaro-o/dev/compare_population_and_tweet_number/outputs/perhour/table_holiday.csv')

In [91]:
tmp = np.load(University_tweets_PATH)

In [92]:
tmp

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])